In [ ]:
%pip install --upgrade pip
%pip install --pre -U "weaviate-client==4.*"
%pip install qdrant-client

In [2]:
import random
import json
import weaviate
import weaviate.classes as wvc

from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance, VectorParams
from weaviate.util import generate_uuid5


In [3]:
qdrant_client = QdrantClient(
        url="https://0c82e035-1105-40f2-a0bd-ecc44a016f15.us-east4-0.gcp.cloud.qdrant.io:6333",
        api_key="37wBxvjnYL2WhZ37aH-nQjZmx1apOmwO73J8aJyj7yxSA71LtcPPBA",
)

weaviate_client = weaviate.connect_to_wcs(
    cluster_url="https://1syylnhesmc5shxvmydv1q.c1.us-east4.gcp.weaviate.cloud",
    auth_credentials=weaviate.AuthApiKey("C3kymue0kUBSuoh5tIwo24nzEr7mhxIOtexi")
)

In [4]:
def create_weaviate_collection(collection_name):
    return weaviate_client.collections.create(
        name=collection_name,
        vectorizer_config=wvc.Configure.Vectorizer.text2vec_openai(),
        generative_config=wvc.Configure.Generative.openai()
    )

In [14]:
def migrate_data(collection_name, batch_size=100):
    weaviate_collection = weaviate_client.collections.get(collection_name)
    if weaviate_collection is None:
        weaviate_collection = create_weaviate_collection(collection_name)
    else:
        print("Collection already exists, deleting and recreating")
        weaviate_client.collections.delete(collection_name)
        weaviate_collection = create_weaviate_collection(collection_name)
    offset = 0
    while True:
        print(f"Batch: {offset}")
        # Retrieve a batch of data from Qdrant
        points = qdrant_client.scroll(
            collection_name=collection_name,
            limit=batch_size,
            offset=offset,
            with_payload=True,
            with_vectors=True
        )

        if points[1] is None:
            break

        data_objects = []

        for point in points[0]:
            properties = point.payload
            data_object = weaviate.classes.DataObject(
                vector=point.vector,
                properties=properties,
                uuid=generate_uuid5(point.id),
            )
            data_objects.append(data_object)

        weaviate_collection.data.insert_many(data_objects)
        offset = points[1]

In [15]:
collections = qdrant_client.get_collections()
for collection in collections.collections:
    collection_name = collection.name
    get_collection = qdrant_client.get_collection(collection_name)
    migrate_data(collection_name, 100)

Collection already exists, deleting and recreating
Batch: 0


KeyboardInterrupt: 